In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.stats.proportion as smp
from ipywidgets import interact
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Custom function for creating grouped barplots of the variables of interest and save the output

def custom_barplots(dataset, # dataframe
                    groupping, # string
                    variables_of_interest, # list of string(s)
                    barwidth=0.5, myspacingy=0.09, myspacingx=8, myfontsize=17):
    
    for variable in variables_of_interest:

        temp = dataset.copy()

        # Group according to the features of interest and calculate the % of answer types for the main target feature
        temp = temp.groupby([groupping, variable])[variable].agg(['count'])
        temp['total'] = temp.groupby(groupping)['count'].transform('sum')
        lower, upper = smp.proportion_confint (temp['count'], temp['total'], alpha=0.05, method='normal')
        temp['CI_prop_upper'] = upper
        temp['CI_prop_lower'] = lower
        temp[variable + ' (%)'] = temp['count'] / temp['total'] * 100
        temp['CI_perc_upper'] = temp['CI_prop_upper'] * 100
        temp['CI_perc_lower'] = temp['CI_prop_lower'] * 100
        temp['abs_err'] = temp[variable + ' (%)'] - temp['CI_perc_lower']
        temp.reset_index(inplace=True) # To 'undo' the grouping
        temp = temp.round(1)
        temp
        
        # Recode the answers of those variables we want to use to stratify the population
        answers_to_rename = [variable]
        temp[answers_to_rename] = temp[answers_to_rename].replace(answers_dict)

        abs_err = temp.pivot(index=variable, columns=groupping, values='abs_err')

        g = temp.pivot(index=variable, columns=groupping, values=variable+' (%)')\
        .plot(kind='barh', xerr=abs_err, width=barwidth, figsize=(3,10), colormap='coolwarm')
        for item in ([g.title, g.xaxis.label, g.yaxis.label] + g.get_xticklabels() + g.get_yticklabels()):
            item.set_fontsize(20)
        plt.title(questions_dict.get(variable), x =0,  fontsize = 20)
        plt.xlabel(variable + ' (%)')
        plt.ylabel('')
        plt.xlim(0, max(temp[variable +' (%)'] + 15))
        handles, labels = g.get_legend_handles_labels()
        g.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1.5, 0.75), fontsize=16)
        g.spines['top'].set_visible(False)
        g.spines['right'].set_visible(False)
        g.spines['left'].set_visible(False)

        # To plot the % number
        for p in g.patches:
            width, height = p.get_width(), p.get_height()
            x, y = p.get_xy() 
            g.annotate('{:}%'.format(width), (x + width + myspacingx, y + myspacingy), fontsize = myfontsize)
            # play with fontsize, depending on graph

In [10]:
# Dictionary to recode values (as on survey dataset)
  
# Create a dictionary of question names
questions_dict_path = 'https://raw.githubusercontent.com/lorena-gp/food-standards-agency_app/master/survey_guide_variables.csv'
questions_dict = pd.read_csv(questions_dict_path)
questions_dict = pd.Series(questions_dict.Label.values, index=questions_dict.Variable).to_dict()

# Create a nested dictionary of answer names
answers_dict_path = 'https://raw.githubusercontent.com/lorena-gp/food-standards-agency_app/master/survey_guide_values.csv'
answers_dict = pd.read_csv(answers_dict_path)
answers_dict['Label'] = (answers_dict['Label']
                         .replace({'Wave 1':2010, 'Wave 2':2012, 'Wave 3':2014, 'Wave 4':2016, 'Wave 5':2018})
                         .replace({'Married/Civil Partnership/Living with Partner':'Married/Partnership'})
                         .replace({'Single/Widowed/Divorced/Separated/Other':'Single/Other'}))
answers_dict = answers_dict.fillna(method='ffill')
answers_dict = answers_dict.groupby('Variable')[['Vlue', 'Label']].apply(lambda g: dict(g.values)).to_dict()
answers_dict['wimd_2014_quintile'] = {1: 1, 2: 2, # 1 is most deprived
                                      3: 3, 4: 4, 5: 5, # 5 is least deprived
                                      -8: "Don't know", -1:'Not applicable'}
answers_dict['hhdinc'] = {1: '£10,399 or less', 2: '£10,400 - £25,999', 3: '£26,000 - £51,999', 4:'£52,000 or more',
                          -9:'Refused', -8: "Don't know", -1:'Not applicable'}

In [11]:
# Load Food and You survey dataset
survey_path = 'https://raw.githubusercontent.com/lorena-gp/food-standards-agency_app/master/survey.csv'
survey_full_dataset = pd.read_csv(survey_path)
survey_full_dataset = pd.DataFrame(survey_full_dataset)

# Encode 'Not applicable', 'Refused' and 'Don't know' as NaN
survey_full_dataset = survey_full_dataset.replace([-9, -8, -1, 98], np.nan)
cols_5_NaN = ['q4_1_4', 'q4_1_5a', 'Q4_1_5_comb', 'q4_1_6', 'q4_1_7', 'q4_1_8a', 'q4_1_8b', 'sanspray', 'q4_1_11',
              'q4_1_12', 'q4_1_13', 'q4_1_14', 'q4_1_15', 'q4_1_16', 'q4_1_17', 'q4_1_18', 'q4_1_19']
survey_full_dataset[cols_5_NaN] = survey_full_dataset[cols_5_NaN].replace([5], np.nan)

survey_full_dataset['age_dv_grouped'] = (survey_full_dataset['age_dv']
                                         .replace({1:'A', 2:'B', 3:'B', 4:'B', 5:'C', 6:'C', 7:'C'})
                                         .replace({'A':'16-24', 'B':'25-54', 'C':'55+'}))

In [12]:
# Define demographic variables of interest
demographic_variables = ['age_dv', 'marstat2', 'religion_dv', 'RespSex', 'wimd_2014_quintile', 'workstat2',
                         'Q6_1', 'age_dv_grouped', 'UrbanRuralInd', 'bhhsize2', 'hhdinc',
                         'below6', 'below16']

In [13]:
# Define questions of interest

# General knowledge based
questions_of_interest = ['bpoison', 'Q4_1_5_comb', 'eatoutev', 'q4_1_6', 'Q4_1_5_comb', 'Q4_19b', 'fdsecst',
                         'sanspray', 'Q4_26b', 'q4_1_8a', 'q4_1_11', 'dq4_1bc', 'Q4_143', 'safemeat10',
                         'Q4_2610', 'q4_27_4_slice', 'Q4_27c', 'Q4_28b8', 'q2_14s10a_dv', 'q2_14su8b_dv',
                         'q2_14su7a_dv', 'q2_14s13_dv', 'EatOut1', 'EatOut2', 'EatOut3']

# Correlation analysis by James
questions_high_correlation_1 = ['clinaller', 'FdAuthAct_MC9', 'chemknow3', 'Q3_32', 'Q4_1b1', 'safeegg7', 'chemknow2',
                              'q2_14s15_dv', 'EatOutInfDV_Wb', 'chemiop3', 'Q4_1b4',  'Q4_24, provfd1', 'Q3_1312',
                              'Q3_135', 'chemknow4', 'q2_14s10_dv', 'Q4_266', 'q2_14su8a_dv', 'Q11_8bDV17',
                              'chemiop2', 'Q11_8bDV2', 'FdAuthAct_MC2', 'Q3_137', 'Q4_114', 'Worried', 'Q11_8bDV12',
                              'q4_1_15', 'chemiop1', 'q4_277dv', 'fdreac_dv','chemknow1', 'FdAuthAct_MC6',
                              'Q2_38DV5', 'Q4_115', 'reacno', 'heardorgc9', 'Q4_113', 'Q11_8bDV10', 'safefish5',
                              'EatOutInfDV_Ad', 'safeegg14', 'Q2_19', 'Q2_35DV_Pr', 'Q4_2611', 'q4_276dv', 'Q4_2610',
                              'q4_277_dv', 'Q4_143', 'Q11_8bDV6', 'heardorgc6', 'q4_276_dv', 'q2_14su7_dv', 'Q4_151',
                              'safecheese5', 'safecheese2', 'Q4_227']

# Correlation analysis by Lorena
questions_high_correlation_2 = ['q4_276dv', 'q4_276_dv', 'fdreac_dv', 'reacno', 'q2_14s10_dv', 'FdAuthAct_MC9',
                                'Q4_28b5', 'q4_277dv', 'q2_14su5_dv', 'Q11_8bDV12', 'Q2_35DV_Rv', 'heardorgc1',
                                'heardorgc3', 'Q4_1b3', 'heardorgc5', 'EatOutInfDV_Wb', 'hhdinc', 'EatOut1',
                                'heardorgc4','q4_278dv', 'q4_1_15', 'HeardFSA', 'heardorgc6']

questions_of_interest = questions_of_interest + questions_high_correlation_1 + questions_high_correlation_2

questions_of_interest = list(set(questions_of_interest)) # Get unique questions

In [14]:
# To consider only the survey answers from wave 4 and 5 only:
waves = [4,5]
survey_subpopulation = survey_full_dataset.loc[survey_full_dataset['surveyyear'].isin(waves)]
survey_subpopulation[demographic_variables] = survey_subpopulation[demographic_variables].replace(answers_dict)

# Interactive dashboard 

## To give access to voila dashboard remotely using github and binder:

- Set up a public github repo with all data files needed (as csv), jupyter notebook with widgets (in which all csv files are read using the raw URLs from github) and a requirements.txt (listing  all the modules needed to run the notebook).
- Use binder (https://mybinder.org) and specify the path of the jupypter notebook (using voila/render), indicate it is a URL path. After this, a long URL adress is generated within the mybinder set up box: copy this and save, as this is the safest way to share the app afterwards.
- The app will open in its own URL adress, which can also be shared, but tends to expired withing a few hours.
- In general, the loading of the app with either of the two URLs above can fail at times. Trying a different browser is recomended. If possible persist, process to generate the app again using https://mybinder.org, as this is a fairly easy and quick set up anyway.

## Dashboard can be executed within the jupyter notebook
- It appears below the cell with the execution code

## To show this as an independent (local) dashboard with Voila 
(https://github.com/voila-dashboards/voila):
Install in terminal by: conda install -c conda-forge voila
Start Voilà locally (cd in directory with this notebook) by running: voila Food-and-You-survey_risks.ipynb


## To give access to voila dashboard remotely using ngrok:

- See https://voila.readthedocs.io/en/stable/deploy.html#sharing-voila-applications-with-ngrok)
- Install ngrok: https://ngrok.com/download, unzip file and, if using macOS, move executable file to /usr/local/bin
- Start Voilà locally (cd in directory with this notebook) by running: voila Food-and-You-survey_risks.ipynb
- In a new terminal window, start ngrok by running: ngrok http 8866 (check local host number actually used by the dashboard of interest, as, if running voila multiple times, this number will change)
- Copy the link from the ngrok terminal window (link looks like https://8bb6fded.ngrok.io) and use or send link.

### Notes

- Website will take some time to load, as the jupyter noteook is running in the background.
- When using the ngrok link, the requests will be forwared to your local instance of Voilà.
- Census data is too heavy and cannot be ploted as a voila dashboard

In [18]:
# Set up for the interactive dashboard inside notebook:
# Plot food risks associated to desired subpopulation groups for specified relevant questions:

country_dict = answers_dict['country_dv']
country_list = list(survey_subpopulation.country_dv.unique())
country_list[:] = [country_dict.get(e,'') for e in country_list]
country_dict_inv = {v: k for k, v in country_dict.items()}

region_dict = answers_dict['region_dv']
region_list = list(survey_subpopulation.region_dv.unique())
region_list[:] = [region_dict.get(e,'') for e in region_list]
region_dict_inv = {v: k for k, v in region_dict.items()}

questions_of_interest_names = questions_of_interest.copy()
questions_of_interest_names[:] = [questions_dict.get(e,'') for e in questions_of_interest_names]
questions_dict_inv = {v: k for k, v in questions_dict.items()}



demographics_list = ['Not applicable', 'Age (in detail)', 'Age', 'Gender', 'Marital status', 'Working status',
                     'Religion', 'Health', 'Urban-Rural classification', 'Household size',
                     'Household income', 'Children under 6 in household', 'Children under 16 in household']

demographic_dict = {'Not applicable':'dummy', 'Age (in detail)':'age_dv', 'Age':'age_dv_grouped', 'Gender':'RespSex',
                    'Marital status':'marstat2', 'Working status':'workstat2', 'Religion':'religion_dv',
                    'Health':'Q6_1', 'Urban-Rural classification':'UrbanRuralInd',
                    'Household size':'bhhsize2', 'Household income':'hhdinc',
                    'Children under 6 in household':'below6', 'Children under 16 in household':'below16'}

## You can explore the results of the "Food and you" survey (2016, 2018), for the whole UK, below:

In [19]:
@interact
def plot(question=questions_of_interest_names,
         demographics1=demographics_list,
         demographics2=demographics_list,
         demographics3=demographics_list):
    
    survey_subpopulation['dummy']=''
    survey_subpopulation['combined_demographics']=(survey_subpopulation[demographic_dict.get(demographics1)]+' '+
                                                   survey_subpopulation[demographic_dict.get(demographics2)]+' '+
                                                   survey_subpopulation[demographic_dict.get(demographics3)])        
    custom_barplots(survey_subpopulation,
                    'combined_demographics',
                    [questions_dict_inv.get(question)],
                    myspacingy=0.03, myspacingx=15, barwidth=0.9)

interactive(children=(Dropdown(description='question', options=('Have you had food poisoning in the last year?…

## You can explore the results of the "Food and you" survey (2016, 2018), for each UK country, below:

In [20]:
@interact
def plot(country=country_list,
         question=questions_of_interest_names,
         demographics1=demographics_list,
         demographics2=demographics_list,
         demographics3=demographics_list):
    
    survey_subpopulation['dummy']=''
    survey_subpopulation['combined_demographics']=(survey_subpopulation[demographic_dict.get(demographics1)]+' '+
                                                   survey_subpopulation[demographic_dict.get(demographics2)]+' '+
                                                   survey_subpopulation[demographic_dict.get(demographics3)])        
    custom_barplots(survey_subpopulation[survey_subpopulation.country_dv.eq(country_dict_inv.get(country))],
                    'combined_demographics',
                    [questions_dict_inv.get(question)],
                    myspacingy=0.03, myspacingx=15, barwidth=0.9)

interactive(children=(Dropdown(description='country', options=('England', 'Wales', 'Northern Ireland'), value=…

## You can explore the results of the "Food and you" survey (2016, 2018), for each UK region, below:

In [21]:
@interact
def plot(region=region_list,
         question=questions_of_interest_names,
         demographics1=demographics_list,
         demographics2=demographics_list,
         demographics3=demographics_list):
    
    survey_subpopulation['dummy']=''
    survey_subpopulation['combined_demographics']=(survey_subpopulation[demographic_dict.get(demographics1)]+' '+
                                                   survey_subpopulation[demographic_dict.get(demographics2)]+' '+
                                                   survey_subpopulation[demographic_dict.get(demographics3)])        
    custom_barplots(survey_subpopulation[survey_subpopulation.region_dv.eq(region_dict_inv.get(region))],
                    'combined_demographics',
                    [questions_dict_inv.get(question)],
                    myspacingy=0.03, myspacingx=15, barwidth=0.9)

interactive(children=(Dropdown(description='region', options=('South West', 'South East', 'Wales', 'Yorkshire …